In [ ]:
import os
import cv2
import numpy as np

import numpy as np
import matplotlib.pyplot as plt

import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from torchvision.transforms import v2

from reader import read_data

In [ ]:
#ИНС обучалась на данных размером 128х128, для этого на входе сети также установлено преобразование к этому формату
transforms = v2.Compose([
    v2.Resize((128, 128)),
])

В следующей ячейке организованы циклы обучения и валидации

In [ ]:
def train_loop(dataloader, model, criterion, optimizer):
    num_batches = len(dataloader)

    train_loss = 0

    for imgs, labels in dataloader:
        # Compute prediction and loss
        pred = model(imgs.to(device))
        loss = criterion(pred, labels.to(device))

        # Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= num_batches
    print(f"Train loss: {train_loss:>8f}")

    return train_loss


def test_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for imgs, labels in dataloader:
            # Compute prediction and loss
            pred = model(imgs.to(device))
            loss = criterion(pred, labels.to(device))

            test_loss += loss.item()
            #print(pred, pred.argmax(1))
            #print(labels, labels.argmax(1))
            correct += (
                (pred.round() == labels.to(device)).type(torch.float).sum().item()
            )

    test_loss /= num_batches
    correct /= size
    print(f"Val loss: {test_loss:>8f}, val accuracy: {(100*correct):>0.1f}% \n")

    return test_loss

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Данные для обучения были предварительно подготовлены: разделены на три подвыборки с учётом их ориджина, а также приведены к размеру 128х128

In [ ]:
PATH = "./"
train_path = "train/"
val_path = "val/"
test_path = "test/"

x_train, y_train, x_val, y_val, x_test, y_test = read_data(train_path, val_path, test_path)
#y_test.sum(dim=0)

Для обучения была выбрана архитектура, имеющая вид энкодер + полносвязные слои, что позволяет выделить признаки с изображения, а затем на их основе провести классификацию. Выбор был остановлен на стуктуре NeuralNetwork1

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=1, kernel_size=64, padding='same'),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=32, padding='same'),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(in_features=32**2, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = transforms(x)
        logits = self.layers_stack(x)
        return logits

In [ ]:
class NeuralNetwork1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=1, kernel_size=64, padding='same'),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=32, padding='same'),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=32, padding='same'),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(in_features=16**2, out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.layers_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

Для обучения использовался оптимизатор Adam и бинарная кросс-энтропия

In [ ]:
model = NeuralNetwork1().to(device)

num_epochs = 200
batch_size = 64
learning_rate = 1e-5


train_dataloader = DataLoader(TensorDataset(x_train,y_train), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(TensorDataset(x_val,y_val), batch_size=batch_size, shuffle=False)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.7, 0.999), eps=1e-08)
scheduler = StepLR(optimizer, step_size=80, gamma=0.5)

In [ ]:
loss_history = {"train": [], "test": []}

for i in range(num_epochs):
    print(f"Epoch {i+1}")
    train_loss = train_loop(train_dataloader, model, criterion, optimizer)
    test_loss = test_loop(test_dataloader, model, criterion)

    scheduler.step()

    loss_history["train"].append(train_loss)
    loss_history["test"].append(test_loss)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), loss_history["train"], label="train")
plt.plot(range(1, num_epochs + 1), loss_history["test"], label="test")
plt.xlabel("Epochs", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.legend()
plt.grid()
plt.show()

In [ ]:
В следующей ячейке выводится значение accuracy для классификации на тестовой выборке. 
В сохраннённой модели точность достигла 85%.
Применение точности в данной случае возомжно ввиду того, что соотношение классов близко к 1:1. 

In [ ]:
((model(x_test.to(device)).round() == y_test.to(device)).type(torch.float).sum().item() / len(x_test)) * 100

In [ ]:
#torch.save(model.state_dict(), PATH+'model_state.pth')